In [ ]:
import os
# define the directories where the imgaes are
pizza_directory="./pizza/"
not_pizza_directory="./not_pizza/"

# get the images from the directories
not_pizza_files=[[0, not_pizza_directory, i] for i in os.listdir(not_pizza_directory)]
pizza_files=[[1, pizza_directory, i] for i in os.listdir(pizza_directory)]

print("Number of pizza images:",len(pizza_files))
print("Number of non-pizza images:",len(not_pizza_files))

# combine the image sets
all_files = not_pizza_files + pizza_files

print("Total number of images:", len(all_files))

# each element in all_files is of the format [is_pizza: boolean, directory: string, filename: string]
# in other words, for each `img` in `all_files`:
#    - img[0] = 0 if not pizza, 1 if pizza
#    - img[1] = directory where the image is
#    - img[2] = file name

In [ ]:
from PIL import Image

import numpy as np

standardSize = (512,512)
resizedImages = list()
isPizzaArray = list()

# structure is:
# resizedImages[i] = 2-D numpy array of pixels, [512][512][3]


PATH = os.path.abspath(os.getcwd())
if (not os.path.exists(f'{PATH}/resizedImages')):
    os.mkdir('resizedImages')

for i, image in enumerate(all_files):
    imgData = Image.open(image[1] + image[2])
    imgData = imgData.resize(standardSize)
    imgData.save(f'{PATH}/resizedImages/{i}.jpg')
    # storing images
    resizedImages.append(np.array(imgData))
    isPizzaArray.append(image[0])

X = np.array(resizedImages)
y = np.array(isPizzaArray)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

# ! Takes about 30 seconds...
print("Done")

# Second Model, using an SVM.
To determine best svm kernel we iterated through a random subset of our data and computed the score.

In [ ]:
from sklearn.svm import SVC
# thing=X_trai[0].reshape(1,X_train[0].size)
# thing.shape
X_trai=X_train[0:10]
y_trai=y_train[0:10]
X_tes=X_test[30:50]
y_tes=y_test[30:50]
X_trai=np.array(X_trai.reshape((X_trai.shape[0], 512*512*3)),dtype=np.uint8)
X_tes=np.array(X_tes.reshape((X_tes.shape[0], 512*512*3)),dtype=np.uint8)

for i in ["linear","rbf","poly","sigmoid"]:
    SVM_cl=SVC(kernel=i)
    SVM_cl.fit(X_trai,y_trai)
    X_tes=X_test[30:50]
    y_tes=y_test[30:50]
    print("Mean accuracy of kernel {} {}".format(i,SVM_cl.score(X_tes,y_tes)))
    #Best appears to be sigmoid

# Implement SVM calc, with Naive Bayes
We could not append the svm score of each image to the obseravtion data of the images being fed to the neural net. Because the neural net uses convolution, we could not add more dimensions, or replace one of the tensors with the svm score without causing erronious bias. So instead, we inserted the SVM classification and neural net classification into a Categorical Naive Bayes classifier.

In [ ]:
from keras.layers import Conv2D, Flatten, Dense
from keras.models import Sequential
from sklearn.svm import SVC
from sklearn.metrics import classification_report
np.random.seed(1)

"Preparing data"
X_trai=X_train[0:20]
y_trai=y_train[0:20]
X_tes=X_test[0:10]
y_tes=y_test[0:10]
"Create and Run NN"
model = Sequential() # Initialising the ANN
imgShape = (512, 512, 3)
model.add(Conv2D(7, 3, activation = 'tanh',input_shape=imgShape))
model.add(Conv2D(3, 2, activation = 'relu'))
model.add(Flatten())
model.add(Dense(1,activation="sigmoid")) #Here is the difference. Each pixel will push the image above or below the SVM constraints.
model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy',metrics=["accuracy"])
numEpoches = 3
model.fit(X_trai, y_trai, batch_size = 5, epochs = 3, verbose = 1)
NN_SVM_pred_train=[[np.round(i)] for i in model.predict(X_trai)]
NN_SVM_pred_test=[[np.round(i)] for i in model.predict(X_tes)]

"Create and Run SVM"
X_trai_SVM=np.array(X_trai.reshape((X_trai.shape[0], 512*512*3)),dtype=np.uint8)
X_tes_SVM=np.array(X_tes.reshape((X_tes.shape[0], 512*512*3)),dtype=np.uint8)
SVM_cl=SVC(kernel="sigmoid")
SVM_cl.fit(X_trai_SVM,y_trai)
for i,ii in enumerate(SVM_cl.predict(X_tes_SVM)):
    NN_SVM_pred_test[i].append(ii)
for i,ii in enumerate(SVM_cl.predict(X_trai_SVM)):
    NN_SVM_pred_train[i].append(ii)

In [ ]:

"Create and Run Naive"
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import classification_report
CNB_obj=CategoricalNB()                     #initialized object
CNB_obj.fit(X=NN_SVM_pred_train,y=y_trai)
print(classification_report(y_tes,CNB_obj.predict(NN_SVM_pred_test))) #classification report